# Lab 5: Regression and Regularization

In today’s lab, we'll analyze a dataset from a **servo control system**, which models how different mechanical and electrical components influence the system's response time. A **servo** is a type of motor used in engineering and robotics to precisely control movement. The dataset consists of historical simulation data, where various factors—such as motor type, screw type, and gain settings—impact the time it takes for the system to respond to a step input.

Using this data, we will build a regression model to predict the rise time of the servo mechanism based on its configuration. Understanding how these factors affect response time is crucial in designing efficient and reliable control systems, making this an important problem in engineering and automation.

In [ ]:
# JUST RUN THIS CELL - imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Task 1: Loading in the data

We're going to be using a dataset from the **UCI Machine Learning Repository.** Go to the dataset website [here](https://archive.ics.uci.edu/dataset/87/servo) and follow the steps to **import the dataset in Python**.
- URL also linked for convenience: https://archive.ics.uci.edu/dataset/87/servo

In [ ]:
...

In [ ]:
...

In [ ]:
# JUST RUN THIS CELL - Sanity Check
print(X.shape)
X.head()

In [ ]:
# JUST RUN THIS CELL - Sanity Check
print(y.shape)
y

### Task 2: Understanding our Features

In our dataset `X`, we have 4 features:

- **`motor`**: A categorical variable representing the type of motor used in the servo system. Possible values are **A, B, C, D, and E**.
- **`screw`**: A categorical variable representing the type of screw in the system, with possible values **A, B, C, D, and E**.
- **`pgain`**: An integer representing the proportional gain setting of the servo, which influences how aggressively the system corrects for errors. Possible values range from **3 to 6**.
- **`vgain`**: An integer representing the velocity gain setting, which affects how the system responds to velocity changes. Possible values range from **1 to 5**.

Our goal is to use these features to predict the **rise time** of the servo (the `class` column in our target variable `y`), a continuous numerical value that indicates how quickly the system responds to a step input.

Visualize the distribution of each variable using a **bar plot**: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html

**Hint**: You will need to use `pandas.Series.value_counts()`
- https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html

In [ ]:
# YOUR ANSWER GOES HERE - Visualize the distribution of motor types
...
plt.xlabel("Motor Type")
plt.ylabel("Count")
plt.title("Count of Motor Types for Servo")

In [ ]:
# YOUR ANSWER GOES HERE - Visualize the distribution of screw types
...
plt.xlabel("Screw Type")
plt.ylabel("Count")
plt.title("Count of Screw Types for Servo")

In [ ]:
# YOUR ANSWER GOES HERE - Visualize the distribution of pgain settings
...
plt.xlabel("Proportional Gain Setting")
plt.ylabel("Count")
plt.title("Distribution of Proportional Gain Setting for Servo")

In [ ]:
# YOUR ANSWER GOES HERE - Visualize the distribution of vgain settings
...
plt.xlabel("Velocity Gain Setting")
plt.ylabel("Count")
plt.title("Distribution of Velocity Gain Setting for Servo")

### Task 3: Visualizing Our Target

The goal of our regression model is going to be to predict rise time `y`. Create a **scatterplot** (https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html) of:
- `pgain` vs `y` and print the correlation coefficient (https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)
- `vgain` vs `y` and print the correlation coefficient (https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)


In [ ]:
# YOUR ANSWER GOES HERE - Scatterplot of pgain vs. class
...

In [ ]:
# YOUR ANSWER GOES HERE - Scatterplot of vgain vs. class
...

You might notice that the scatterplots look a little weird. That's because even though the `pgain` and `vgain` variables are integers (numeric), they are actually *discrete* values, and there are only 4 possible `pgain` values and 5 possible `vgain` values.

A better way to visualize this data might be doing side-by-side box plots so that we can visualize the distribution of servo rise time within each discrete bucket.

In [ ]:
# JUST RUN THIS CELL

df = X.copy()
df["class"] = y["class"]

# Create subplots for pgain and vgain
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Box plot for pgain
df.boxplot(column='class', by='pgain', ax=axs[0])
axs[0].set_title('Rise Time Distribution by Pgain')
axs[0].set_xlabel('Pgain')
axs[0].set_ylabel('Rise Time')
axs[0].grid(True)

# Box plot for vgain
df.boxplot(column='class', by='vgain', ax=axs[1])
axs[1].set_title('Rise Time Distribution by Vgain')
axs[1].set_xlabel('Vgain')
axs[1].set_ylabel('Rise Time')
axs[1].grid(True)

# Adjust layout and remove automatic figure title
plt.suptitle('')
plt.tight_layout()
plt.show()

In [ ]:
# JUST RUN THIS CELL

# Create subplots for pgain and vgain
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Box plot for pgain
df.boxplot(column='class', by='motor', ax=axs[0])
axs[0].set_title('Rise Time Distribution by Motor Type')
axs[0].set_xlabel('Motor Type')
axs[0].set_ylabel('Rise Time')
axs[0].grid(True)

# Box plot for vgain
df.boxplot(column='class', by='screw', ax=axs[1])
axs[1].set_title('Rise Time Distribution by Screw Type')
axs[1].set_xlabel('Screw Type')
axs[1].set_ylabel('Rise Time')
axs[1].grid(True)

# Adjust layout and remove automatic figure title
plt.suptitle('')
plt.tight_layout()
plt.show()

### Comment on your findings below. What can you say about the shape, center, and spread of the distribution of Rise Times with respect to each feature? Which features have the most prominent outliers?

(YOUR ANSWER GOES HERE.)

### Task 4: One-Hot Encoding

In order to include categorical variables like `motor` and `screw` types into a model that can only take in numerical inputs, we need to use **one-hot encoding**.

Imagine we are trying to predict someone's final grade in CS 61A based on whether or not they passed or failed Midterm 1, using the regression model:

$$\text{Final Grade Prediction} = \hat{y}_i = \hat{\alpha} + \hat{\beta} \cdot \text{Pass/Fail}_i$$
    
Pass/Fail here would be a categorical variable. However, we can treat this as sort of a "boolean" indicator variable, encoding the "passes" as 1 and the "fails" as 0. In other words, we can specify our prediction model as:

$$\text{Final Grade Prediction} = \hat{y}_i = \hat{\alpha} + \hat{\beta} \cdot \{\text{1 if Student Passed, 0 if Student Failed}\}_i$$

Notice here that we only needed **one indicator variable** to encode **two categories**. That's because the single indicator fully captures information about both categories, since if the indicator is 0 we know that the student failed the midterm. We don't need to add a separate indicator that equals 1 if the student failed because that would make our model redundant.

You might argue that a binary Pass/Fail indicator is not complex enough of a predictor to capture patterns in a student's final grade. However, the 61A course staff informs you that due to privacy concerns, they will not release students' actual midterm scores for your model. So you take a different approach – you send out a survey with 3 options:
- If a student did more than 5 points above the mean
- If a student did more than 5 points below the mean
- If a student got within 5 points of the mean

Now we have 3 categories we need to encode. How do we do this? Well, recall that to encode two categories like Pass/Fail, we only needed 1 indicator to fully encode information about both categories. Similarly, to **encode 3 categories**, we only need **2 indicators** to fully capture all the information. Let's make this explicit with an example model:

$$\text{Final Grade Prediction} = \hat{y}_i = \hat{\alpha} + \hat{\beta} \cdot \{\text{1 if MT 1 Score > Mean + 5, 0 Otherwise}\}_i + \hat{\gamma} \cdot \{\text{1 if MT 1 Score < Mean - 5, 0 Otherwise}\}_i$$

If a student did not score more than 5 points above the mean, and the student did not score less than 5 points below the mean, then by process of elimination, we know that the student did within 5 points of the mean. Therefore, we don't need a third indicator variable to encode that information; it would make our model redundant.

In general, this gets at a very important property of one-hot encoding: **if you are trying to encode $k$ categories, you only need $k - 1$ indicator variables (also called "dummies") to encode all of them.**

Now for the coding task: use `pandas.get_dummies()` to do one-hot encoding for the categories in `motor` and `screw`:
- https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

Some important parameters to pay attention to:
- the `prefix` parameter makes it so that when you generate the new indicator columns, their column names will start with whatever prefixes you assign, which makes it easier to refer to them later (you remember which original variable they were encoded from if you pass in the original column names, for example)
- the `columns` parameter allows you to specify the names of the columns you want to one-hot encode (by default, if this isn't specified, it will encode all object/String/category data types, but in some cases we may want to encode numerically stored categories...which can be argued is the case for our problem, but we won't bother encoding those two columns)
- the `drop_first` is **very important**! You should set this `=True` if you want to ensure you only get $k - 1$ dummies for the $k$ categories you are trying to encode. Otherwise, it will generate a column for each category, which, if you're not careful, may accidentally get passed into your model later.
- the `dtype` parameter will allow you to save the newly encoded columns as `int`s or `float`s if you specify it as such; by default, it will create boolean indicator columns, which we don't necessarily want

Save your newly one-hot encoded data as a dataframe called `design_matrix`.

In [ ]:
encode_cols = ...
design_matrix = ...
design_matrix

### Task 5: Train-Test Split, Fit, Predict

Say it with me everyone: in machine learning, the way we evaluate whether a model is "good" or not is **whether the model is able to generalize to data it hasn't seen before!**

To create "data the model hasn't seen before", we need to create a **training set** – which is data the model will try to learn the underlying pattern behind ("studying for the test") – and a **test set** – which is data the model hasn't been trained on and therefore our way of knowing whether the model can predict well on data it hasn't seen before ("taking the test").

Take a look at `sklearn`'s `train_test_split()` documentation.
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

#### Task 5.1 - Machine Learning Model with 1 Predictor

Using just the `pgain` variable in `design_matrix`, your task is to
- create a train-test split where `X_train` and `X_test` only contain `pgain`
- use `sklearn`'s `LinearRegression` model to fit to the training data. Calculate the training root mean-squared error (RMSE).
    - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
- use your trained `LinearRegression` model to make predictions on the test data. Calculate the test root mean-squared error (RMSE).

In [ ]:
from sklearn.model_selection import train_test_split

X_1 = ... # YOUR ANSWER GOES HERE - Select ONLY pgain from design_matrix
... # YOUR ANSWER GOES HERE - Create a train-test split

In [ ]:
from sklearn.linear_model import LinearRegression

reg = ... # YOUR ANSWER GOES HERE - Fit a linear model using sklearn
y_train_preds = # YOUR ANSWER GOES HERE - Make predictions on the TRAINING set (we will use this later to get the training RMSE)

In [ ]:
def calculate_rmse(y_true, y_preds):
    return ... # YOUR ANSWER GOES HERE - Complete the RMSE Function

training_error = calculate_rmse(...) # YOUR ANSWER GOES HERE - What should we pass in to get training RMSE?
print(training_error)

In [ ]:
y_test_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the TEST set (we will use this later to get the test RMSE)
test_error = ... # YOUR ANSWER GOES HERE - Calculate the test RMSE
print(test_error)

#### Task 5.2 - Machine Learning Model with 2 Predictors

Repeat your steps in Task 5.1, except your design matrix of features should include both `pgain` and `vgain`.

In [ ]:
X_2 = ... # YOUR ANSWER GOES HERE - Select pgain and vgain from design_matrix
... # YOUR ANSWER GOES HERE - Create a new train-test split with these two features

In [ ]:
reg = ... # YOUR ANSWER GOES HERE - Fit a linear model using sklearn
y_train_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the TRAINING set (we will use this later to get the training RMSE)

In [ ]:
training_error = ... # YOUR ANSWER GOES HERE - Calculate the training RMSE
print(training_error)

In [ ]:
y_test_preds =  ... # YOUR ANSWER GOES HERE - Make predictions on the TEST set
test_error = ... # YOUR ANSWER GOES HERE - Calculate the test RMSE
print(test_error)

#### Task 5.4 - Machine Learning Model with All Predictors

Repeat your steps in Tasks 5.1 and 5.2, but this time include all the feature – including the one-hot encoded categorical features – in the `design_matrix`.

In [ ]:
... # YOUR ANSWER GOES HERE - Create a train-test split using all features in design matrix

In [ ]:
reg = ... # YOUR ANSWER GOES HERE - Fit a linear model using sklearn
y_train_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the TRAINING set

In [ ]:
training_error = ... # YOUR ANSWER GOES HERE - Calculate the training RMSE
print(training_error)

In [ ]:
y_test_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the TEST set
test_error = ... # YOUR ANSWER GOES HERE - Calculate the test RMSE
print(test_error)

#### Task 5.5 - What do you notice about the training and testing error as we added more features? Comment on your findings.

(YOUR ANSWER GOES HERE.)

### Task 6 - An Experiment in Nonsense

By the almighty principle of induction, we might be inclined to believe that adding more and more features will keep reducing our training and test error? But is that true? Let's verify it empirically.

For this task, work in groups to do the following:
- set a random seed of 1 using `np.random.seed(1)`
- create two lists or arrays to hold all your training and test RMSEs
- create a feature with randomly generated data. How you randomly generate this data is up to you; it doesn't really matter.
    - one way to generate random data is to draw samples from a standard normal distribution using `np.random.normal()`: https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html
    - again, you need not use a normal distribution. You can randomly generate any numbers you want in any manner you want.
    - make sure your randomly generated feature has the same length as the columns of the design matrix!
- repeat the steps from Task 5 (train-test split, fit, predict) and save your training RMSEs and testing RMSEs in your two lists
- repeat this procedure $n - p$ times, where $n$ is the number of observations in `X_train` and $p$ is the number of features that already exist in `design_matrix`.

In [ ]:
... # YOUR ANSWER GOES HERE - Set a random seed of 1
training_rmses = ... # YOUR ANSWER GOES HERE - Create an empty list or array to hold training RMSEs
testing_rmses = ... # YOUR ANSWER GOES HERE - Create an empty list or array to hold test RMSEs


X_train_master, X_test, y_train, y_test = train_test_split(design_matrix, y, test_size=0.33, random_state=42)

p = len(X_train_master.columns)
design_matrix_copy = design_matrix.copy() # USE THIS COPIED DATAFRAME design_matrix_copy

for i in range(len(X_train_master) - p):
    random_feature = ... # YOUR ANSWER GOES HERE - Create a random feature of your choice
    design_matrix_copy[f"rand_{i+1}"] = random_feature # We add this random feature to a new column
    
    X_train, X_test, y_train, y_test = ... # YOUR ANSWER GOES HERE - Create a new train-test split with this added feature!
    
    reg = ... # YOUR ANSWER GOES HERE - Initialize and fit a linear model from sklearn
    y_train_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the training set
    training_error = ... # YOUR ANSWER GOES HERE - Calculate training RMSE
    y_test_preds = ... # YOUR ANSWER GOES HERE - Make predictions on the test set
    test_error = ... # YOUR ANSWER GOES HERE - Calculate test RMSE
    
    ... # YOUR ANSWER GOES HERE - Append this iteration's training error to your original list or array
    ... # YOUR ANSWER GOES HERE - Append this iteration's test error to your original list or array

Now, let's create a plot of our training error over each iteration:

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(training_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, training_rmses)
plt.xlabel('Iteration')
plt.ylabel('Training Error')
plt.title('Training Error Over Iterations')
plt.grid(True)
plt.show()

Now, let's do the same for the test error.

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(testing_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, testing_rmses)
plt.xlabel('Iteration')
plt.ylabel('Test Error')
plt.title('Test Error Over Iterations')
plt.grid(True)
plt.show()

### What do you observe? What is this phenomenon called? Why is it happening?

(YOUR ANSWER GOES HERE.)

### Task 7 - Bias-Variance Tradeoff

You might often see overfitting and underfitting discussed in the context of the **bias-variance tradeoff**, which helps explain how model complexity affects prediction error. 

- When a model is too simple (low complexity), it has **high bias**. It makes strong assumptions about the data and fails to capture patterns, leading to underfitting.
- On the other hand, when a model is too complex, it has **high variance**. It captures noise along with the true pattern, making it sensitive to small fluctuations in the training data and leading to overfitting.
- The total prediction error consists of bias, variance, and irreducible error. As we add more features or increase model complexity, bias decreases (model fits the data better), but variance increases (model becomes more sensitive to fluctuations).

The goal is to find the optimal model complexity where both bias and variance are balanced, minimizing total error.

![Bias Variance Tradeoff](bias_variance_tradeoff.png)

### YOUR TASK: Explain the bias variance tradeoff in the context of our experiment in Task 6.

(YOUR ANSWER GOES HERE.)

### Task 8: Regularization to the Rescue!

Overfitting is highly problematic for machine learning models; it goes against our main criteria to decide whether a model is "good" or not since the model cannot generalize outside of the data it was trained on. As we have seen, when models become too complex with too many parameters, this can cause the model to learn the irrelevant noise in the training data rather than focus on the general pattern.

One way to combat this is **regularization**, which can be thought of as **making our complex model simpler**. Regularization achieves this by adding a constraint on the parameters (coefficients) of the model and penalizing their magnitudes if they get too large. That way, for our nonsense features created above, their coefficients would be shrunk towards 0 so they do not have has large an impact on the model.

Formally, a regularized linear regression model over $n$ observations and $p$ predictors takes the form:

$$\text{Least Squares Problem} + \alpha * \text{(Regularization ``Penalty" Term)} = \frac{1}{n} \sum_{i=1}^n (y_i - (\sum_{j=1}^p \beta_j x_{ij}))^2 + \alpha \|\beta\|_P^P$$

where $\alpha$ controls the **strength** of the penalty.
- High $\alpha$ values have **larger** penalties on the coefficients and shrink more of them
- Low $\alpha$ values have **smaller** penalties on the coefficients and don't reduce them as much

In the context of optimization, you can think of the least squares problem and the penalty term as **playing a game** against each other. The least square problem really wants to minimize the mean-squared error to get as close as possible to the actual true coefficient values (low bias). The penalty term hates the mean-squared error, and isn't happy when the coefficients are close to the true values. To make both the least squares term and the penalty term happy, the solution to the regularized model will keep the most important least-squares coefficients, but shrink the unimportant or nonsense coefficients down to 0. In effect, we're introducing a little bit of **bias** into the model parameters, but with the benefit that it will **reduce the variance** and help the model generalize well outside the training data with reduced complexity.

#### 8.1 - Exploring $\ell^P$-Norms

One thing you may have noticed in the penalty term of our regularized linear model is the $\|\beta\|_P$ term. This is called an **$\ell^P$-norm**.
- I'm using an uppercase P to avoid confusion with $p$ which is the number of features in our model, but you may see it written as lowercase $\ell^p$-norm in a linear algebra textbook.

Generally, the $\ell^P$ norm of a vector $\beta$ takes the form $\|\beta\|_P = (|\beta_1|^P + |\beta_2|^P + ... + |\beta_p|^P)^{1/P}$

This idea leads to two **very important** types of regularization for linear models:

#### (1) The $\ell^1$-Norm, aka **LASSO Regression**

When P = 1, $\|\beta\|_P^P = \|\beta\|_1^1 = ((|\beta_1|^1 + |\beta_2|^1 + ... + |\beta_p|^1)^{1/1})^1 = \sum_{j=1}^p |\beta_j|$

As we can see, **LASSO Regression** imposes a penalty on the coefficients of our model by summing over their absolute values. Let's see what this constraint looks like in two-dimensions.

First, let's run a least-squares regression on some random data.

In [ ]:
# JUST RUN THIS CELL
np.random.seed(42)
n_samples = 100

# Create two correlated features
X1 = np.random.randn(n_samples)
X2 = 0.5 * X1 + np.random.randn(n_samples) * 0.1  # Slight correlation
X = np.column_stack((X1, X2))

# Generate target variable with only X1 contributing significantly
beta_true = np.array([3, 0])  # Only first feature is important
y = X @ beta_true + np.random.randn(n_samples) * 10

# Fit Ordinary Least Squares (OLS) Regression
ols_model = LinearRegression()
ols_model.fit(X, y)
ols_coeffs = ols_model.coef_
print(ols_coeffs)

Let's plot these points on our 2D Plane.

In [ ]:
# JUST RUN THIS CELL
plt.figure(figsize=(8, 8))
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')
plt.xlim(-8, 8)
plt.ylim(-8, 8)
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.grid()

Now, let's add our $\ell^1$ constraint. In the x-y plane, this will take the form:

$$(|x|^P + |y|^P)^{1/P} = |x| + |y|$$ 

since P = 1. Before running the code below, what shape will this generate?

In [ ]:
# Create the plot centered at the origin with L1 constraint
plt.figure(figsize=(8, 8))

# Plot the Least Squares Solution
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')

# Add L1 constraint
diamond_x = np.array([1, 0, -1, 0, 1]) * 4.05377455
diamond_y = np.array([0, 1, 0, -1, 0]) * 4.05377455
plt.plot(diamond_x, diamond_y, color='gray', label=r'L1 Constraint ($\ell^1$-norm)')

# Set axis limits to match the given plot
plt.xlim(-8, 8)
plt.ylim(-8, 8)

# Add grid, labels, and legend
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.title('Least Squares Solution and L1 Constraint')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

What the $\ell^1$ constraint is essentially doing is creating a region along which the penalized LASSO solution must occur. Basically, all solutions subject to the $\ell^1$ penalty are constrained to that region on the x-y plane.

Now, let's try fitting a **LASSO regression** using `sklearn`. Documentation is available below, but for today's lab, we've done it for you!
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
# JUST RUN THIS CELL
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=1)
lasso_model.fit(X, y)
lasso_coeffs = lasso_model.coef_
print(lasso_coeffs)

**What do you notice about one of the coefficients?**

Let's plot this point along our LASSO constraint.

In [ ]:
# JUST RUN THIS CELL

# Create the plot centered at the origin with L1 constraint
plt.figure(figsize=(8, 8))

# Plot the Least Squares Solution
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')

# Plot the LASSO Solution
plt.scatter(lasso_coeffs[0], lasso_coeffs[1], label='LASSO Solution')

# Add L1 constraint (diamond shape)
l1_radius = np.sum(np.abs(lasso_coeffs))  # Compute the L1 norm of LASSO coefficients
diamond_x = np.array([1, 0, -1, 0, 1]) * l1_radius
diamond_y = np.array([0, 1, 0, -1, 0]) * l1_radius
plt.plot(diamond_x, diamond_y, color='gray', label=r'L1 Constraint ($\ell^1$-norm)')

# Set axis limits to match the given plot
plt.xlim(-8, 8)
plt.ylim(-8, 8)

# Add grid, labels, and legend
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.title('Least Squares Solution and L1 Constraint')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

Let's see what happens when we increase the value of `alpha` – or, in other words, strengthen our penalty term.

In [ ]:
alpha = ...

In [ ]:
# JUST RUN THIS CELL
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X, y)
lasso_coeffs = lasso_model.coef_
print(lasso_coeffs)

In [ ]:
# JUST RUN THIS CELL

# Create the plot centered at the origin with L1 constraint
plt.figure(figsize=(8, 8))

# Plot the Least Squares Solution
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')

# Plot the LASSO Solution
plt.scatter(lasso_coeffs[0], lasso_coeffs[1], label='LASSO Solution')

# Add L1 constraint (diamond shape)
l1_radius = np.sum(np.abs(lasso_coeffs))  # Compute the L1 norm of LASSO coefficients
diamond_x = np.array([1, 0, -1, 0, 1]) * l1_radius
diamond_y = np.array([0, 1, 0, -1, 0]) * l1_radius
plt.plot(diamond_x, diamond_y, color='gray', label=r'L1 Constraint ($\ell^1$-norm)')

# Set axis limits to match the given plot
plt.xlim(-8, 8)
plt.ylim(-8, 8)

# Add grid, labels, and legend
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.title('Least Squares Solution and L1 Constraint')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

What do you observe about our LASSO solution and our constrained region?

#### (2) The $\ell^2$-Norm, aka **Ridge Regression**

LASSO helped us learn what happens when we set P = 1, but really, there's nothing stopping us from increasing P to arbitrary powers.

In addition to LASSO, which is the $\ell^1$-Norm, the most common type of regularization function is the **Ridge penalty** or the $\ell^2$-Norm.

When P = 2, $\|\beta\|_P^P = \|\beta\|_2^2 = ((|\beta_1|^2 + |\beta_2|^2 + ... + |\beta_p|^2)^{1/2})^2 = (\sqrt{\sum_{j=1}^p \beta_j^2})^2 = \sum_{j=1}^p \beta_j^2$

As we can see, **Ridge Regression** imposes a penalty on the coefficients of our model by summing over their squares.

Let's revisit our plots from above, but this time, let's fit a **Ridge regression** to our data and see how it compares to the least squares estimate.
- `sklearn`'s Ridge documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [ ]:
# JUST RUN THIS CELL
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1)
ridge_model.fit(X, y)
ridge_coeffs = ridge_model.coef_
print(ridge_coeffs)

How do these coefficients compare to the LASSO coefficients for the same strength of regularization $\alpha$?

Before we plot the $\ell^2$ constraint in the x-y plane, what do you think the constrained space will look like? In other words, what will

$$(x^P + y^P)^{1/P}$$

look like for P = 2?

In [ ]:
# JUST RUN THIS CELL

# Create the plot centered at the origin with L1 constraint
plt.figure(figsize=(8, 8))

# Plot the Least Squares Solution
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')

# Plot the Ridge Solution
plt.scatter(ridge_coeffs[0], ridge_coeffs[1], label='Ridge Solution')

# Add L2 constraint (circle shape)
theta = np.linspace(0, 2 * np.pi, 100)  # 100 points around the circle
l2_radius = np.sqrt(np.sum(ridge_coeffs ** 2))  # Compute the L2 norm of Ridge coefficients
circle_x = l2_radius * np.cos(theta)  # Parametric equation for a circle
circle_y = l2_radius * np.sin(theta)
plt.plot(circle_x, circle_y, color='gray', label=r'L2 Constraint ($\ell^2$-norm)')

# Set axis limits to match the given plot
plt.xlim(-8, 8)
plt.ylim(-8, 8)

# Add grid, labels, and legend
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.title('Least Squares Solution and L2 Constraint')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

Now, let's once again increase the strength of the regularization parameter, and see what happens both to our constrained space and the coefficients.

In [ ]:
alpha = ...

In [ ]:
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X, y)
ridge_coeffs = ridge_model.coef_
print(ridge_coeffs)

In [ ]:
# JUST RUN THIS CELL

# Create the plot centered at the origin with L1 constraint
plt.figure(figsize=(8, 8))

# Plot the Least Squares Solution
plt.scatter(ols_coeffs[0], ols_coeffs[1], label='Least Squares Solution')

# Plot the Ridge Solution
plt.scatter(ridge_coeffs[0], ridge_coeffs[1], label='Ridge Solution')

# Add L2 constraint (circle shape)
theta = np.linspace(0, 2 * np.pi, 100)  # 100 points around the circle
l2_radius = np.sqrt(np.sum(ridge_coeffs ** 2))  # Compute the L2 norm of Ridge coefficients
circle_x = l2_radius * np.cos(theta)  # Parametric equation for a circle
circle_y = l2_radius * np.sin(theta)
plt.plot(circle_x, circle_y, color='gray', label=r'L2 Constraint ($\ell^2$-norm)')

# Set axis limits to match the given plot
plt.xlim(-8, 8)
plt.ylim(-8, 8)

# Add grid, labels, and legend
plt.xlabel(r'$\beta_1$ (Coefficient for $X_1$)')
plt.ylabel(r'$\beta_2$ (Coefficient for $X_2$)')
plt.title('Least Squares Solution and L2 Constraint')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()

Compare what you are noticing for different values of `alpha` in the Ridge model to what you noticed with different values of `alpha` in the LASSO model.

In general, it takes **much stronger regularization** to shrink the coefficients in the Ridge model compared to LASSO. Additionally, it's much easier for the LASSO model to have coefficients that shrink all the way down to 0, whereas even for large `alpha` in the Ridge model, our coefficients only approach, but never actually reach, 0.

This actually teaches a very valuable lesson in the differences between LASSO and Ridge.
- Because LASSO uses an $\ell^1$ penalty, it is able to induce **sparsity** in its coefficients. In other words, LASSO is able to shrink coefficients all the way down to **exactly 0**.
    - The additional benefit of this is that LASSO also acts as a form of **feature selection** for us! If we are trying to decide which features are important and which aren't, we can use LASSO to shrink the least important features down to 0 and just select a subset of the most important ones.

- Ridge, on the other hand, uses an $\ell^2$ penalty. Because of this, it is much harder for solutions to the Ridge regression to hit exactly 0 on either of the parameters. For high regularization strength, Ridge will shrink the parameters such that they **approach 0**, but they will never actually *equal* 0.

### Task 9: Revisiting Servo with Regularization

Let's go back to our Servo dataset at the beginning of the lab and explore what happens to our models if we use regularization to learn the optimal parameters instead.

In [ ]:
alpha = ...

In [ ]:
# JUST RUN THIS CELL - LASSO Model
X = servo.data.features 
y = servo.data.targets 
encode_cols = ["motor", "screw"]
design_matrix = pd.get_dummies(X, prefix=encode_cols, columns=encode_cols, drop_first=True, dtype=int)
np.random.seed(1)
training_rmses = []
testing_rmses = []
X_train_master, X_test, y_train, y_test = train_test_split(design_matrix, y, test_size=0.33, random_state=42)
p = len(X_train_master.columns)
design_matrix_copy = design_matrix.copy()
for i in range(len(X_train_master) - p):
    random_feature = np.random.normal(size=len(design_matrix_copy))
    design_matrix_copy[f"rand_{i+1}"] = random_feature
    X_train, X_test, y_train, y_test = train_test_split(design_matrix_copy, y, test_size=0.33, random_state=42)
    reg = Lasso(alpha=alpha).fit(X_train, y_train)
    y_train_preds = reg.predict(X_train)
    training_error = calculate_rmse(y_train["class"], y_train_preds)
    y_test_preds = reg.predict(X_test)
    test_error = calculate_rmse(y_test["class"], y_test_preds)
    
    training_rmses.append(training_error)
    testing_rmses.append(test_error)

In [ ]:
reg.coef_

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(training_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, training_rmses)
plt.xlabel('Iteration')
plt.ylabel('Training Error')
plt.title('Training Error Over Iterations')
plt.grid(True)
plt.show()

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(testing_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, testing_rmses)
plt.xlabel('Iteration')
plt.ylabel('Test Error')
plt.title('Test Error Over Iterations')
plt.grid(True)
plt.show()

In [ ]:
alpha = ...

In [ ]:
# JUST RUN THIS CELL - Ridge Model
X = servo.data.features 
y = servo.data.targets 
encode_cols = ["motor", "screw"]
design_matrix = pd.get_dummies(X, prefix=encode_cols, columns=encode_cols, drop_first=True, dtype=int)
np.random.seed(1)
training_rmses = []
testing_rmses = []
X_train_master, X_test, y_train, y_test = train_test_split(design_matrix, y, test_size=0.33, random_state=42)
p = len(X_train_master.columns)
design_matrix_copy = design_matrix.copy()
for i in range(len(X_train_master) - p):
    random_feature = np.random.normal(size=len(design_matrix_copy))
    design_matrix_copy[f"rand_{i+1}"] = random_feature
    X_train, X_test, y_train, y_test = train_test_split(design_matrix_copy, y, test_size=0.33, random_state=42)
    reg = Ridge(alpha=alpha).fit(X_train, y_train)
    y_train_preds = reg.predict(X_train)
    training_error = calculate_rmse(y_train["class"], y_train_preds)
    y_test_preds = reg.predict(X_test)
    test_error = calculate_rmse(y_test["class"], y_test_preds)
    
    training_rmses.append(training_error)
    testing_rmses.append(test_error)

In [ ]:
reg.coef_

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(training_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, training_rmses)
plt.xlabel('Iteration')
plt.ylabel('Training Error')
plt.title('Training Error Over Iterations')
plt.grid(True)
plt.show()

In [ ]:
# JUST RUN THIS CELL
x_values = list(range(11, 11 + len(testing_rmses)))
plt.figure(figsize=(8, 5))
plt.plot(x_values, testing_rmses)
plt.xlabel('Iteration')
plt.ylabel('Test Error')
plt.title('Test Error Over Iterations')
plt.grid(True)
plt.show()

What we've done here is **significantly reduced the variance** of our model by just adding a little bit of bias into our estimated model parameters. In exchange for a little bit of bias, we've greatly reduced the degree to which our model overfits, even for a large number of nonsense features!